# Домашнее задание
- Скачайте dataset iris https://gist.github.com/netj/8836201
- Для каждого типа цветка определите максимальное, минимальное и среднее значение 4-х параметров
- Визуализируйте точечный график (plt.scatter) по каждой паре параметров
- Попробуйте отделить какой-нить тип цветка от всех остальных (сформулируйте правило на основе диаграммы - ЕСЛИ ЗНАЧЕНИЕ ПАРАМЕТРА X БОЛЬШЕ/МЕНЬШЕ Y, ТО ЦВЕТОК СКОРЕЕ ВСЕГО ОТНОСИТСЯ/НЕ ОТНОСИТСЯ К ТИПУ Z)
- Сделайте отдельную колонку для своего предсказания
- Оцените качество (сколько раз Вы угадали с ответом и сколько раз не угадали)

In [ ]:
!pip install pyspark

In [15]:
from pyspark.sql import SparkSession, functions as F

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [37]:
df = spark.read.csv('iris.csv', inferSchema=True, header=True)

In [4]:
df.show(5)

+------------+-----------+------------+-----------+-------+
|sepal.length|sepal.width|petal.length|petal.width|variety|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| Setosa|
|         4.9|        3.0|         1.4|        0.2| Setosa|
|         4.7|        3.2|         1.3|        0.2| Setosa|
|         4.6|        3.1|         1.5|        0.2| Setosa|
|         5.0|        3.6|         1.4|        0.2| Setosa|
+------------+-----------+------------+-----------+-------+
only showing top 5 rows



In [12]:
df.columns

['sepal.length', 'sepal.width', 'petal.length', 'petal.width', 'variety']

In [17]:
cols = [el.replace('.', '_') for el in df.columns]
cols

['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'variety']

In [38]:
df = df.withColumnsRenamed(dict(zip(df.columns, cols)))

In [23]:
df.columns

['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'variety']

In [39]:
l = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
f = ['min'] * 4
d = dict(zip(l, f))
d

{'sepal_length': 'min',
 'sepal_width': 'min',
 'petal_length': 'min',
 'petal_width': 'min'}

In [42]:
df_min = df.groupBy('variety').agg(d)
df_min.show(5)


+----------+----------------+----------------+-----------------+-----------------+
|   variety|min(sepal_width)|min(petal_width)|min(sepal_length)|min(petal_length)|
+----------+----------------+----------------+-----------------+-----------------+
| Virginica|             2.2|             1.4|              4.9|              4.5|
|    Setosa|             2.3|             0.1|              4.3|              1.0|
|Versicolor|             2.0|             1.0|              4.9|              3.0|
+----------+----------------+----------------+-----------------+-----------------+



In [43]:
f = ['max'] * 4
d = dict(zip(l, f))
df_max = df.groupBy('variety').agg(d)
df_max.show(5)

+----------+----------------+----------------+-----------------+-----------------+
|   variety|max(sepal_width)|max(petal_width)|max(sepal_length)|max(petal_length)|
+----------+----------------+----------------+-----------------+-----------------+
| Virginica|             3.8|             2.5|              7.9|              6.9|
|    Setosa|             4.4|             0.6|              5.8|              1.9|
|Versicolor|             3.4|             1.8|              7.0|              5.1|
+----------+----------------+----------------+-----------------+-----------------+



In [52]:
f = ['mean'] * 4
d = dict(zip(l, f))
df_avg = df.groupBy('variety').agg(d)
df_avg.show(5)

+----------+------------------+------------------+-----------------+------------------+
|   variety|  avg(sepal_width)|  avg(petal_width)|avg(sepal_length)| avg(petal_length)|
+----------+------------------+------------------+-----------------+------------------+
| Virginica|2.9739999999999998|             2.026|6.587999999999998|             5.552|
|    Setosa| 3.428000000000001|0.2459999999999999|5.005999999999999|1.4620000000000002|
|Versicolor|2.7700000000000005|1.3259999999999998|            5.936|              4.26|
+----------+------------------+------------------+-----------------+------------------+



In [53]:
for col in df_avg.columns[1:]:
    df_avg = df_avg.withColumn(col, F.round(col, 2))
df_avg.show(5)

+----------+----------------+----------------+-----------------+-----------------+
|   variety|avg(sepal_width)|avg(petal_width)|avg(sepal_length)|avg(petal_length)|
+----------+----------------+----------------+-----------------+-----------------+
| Virginica|            2.97|            2.03|             6.59|             5.55|
|    Setosa|            3.43|            0.25|             5.01|             1.46|
|Versicolor|            2.77|            1.33|             5.94|             4.26|
+----------+----------------+----------------+-----------------+-----------------+



In [54]:
psdf = df.pandas_api()
psdf.head(5)

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


,sepal_length,sepal_width,petal_length,petal_width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [57]:
import plotly.express as px
fig = px.scatter_matrix(psdf.iloc[:, :-1])
fig.show()

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning:

`to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas DataFrame is expected to be small.



In [59]:
fig = px.scatter_matrix(
    psdf
    , dimensions=["sepal_length", "sepal_width", "petal_length", "petal_width"]
    , color='variety'
)
fig.show()

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning:

`to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas DataFrame is expected to be small.



In [66]:
from google.colab import output
output.no_vertical_scroll()

for i in range(psdf.columns[:-1].size):
    x = psdf.columns[i]
    for j in range(i+1, psdf.columns[:-1].size):
        y = psdf.columns[j]
        psdf.plot.scatter(x=x, y=y).show()


<IPython.core.display.Javascript object>

In [67]:
df.show(5)

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|variety|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| Setosa|
|         4.9|        3.0|         1.4|        0.2| Setosa|
|         4.7|        3.2|         1.3|        0.2| Setosa|
|         4.6|        3.1|         1.5|        0.2| Setosa|
|         5.0|        3.6|         1.4|        0.2| Setosa|
+------------+-----------+------------+-----------+-------+
only showing top 5 rows



In [ ]:
df.when()

In [68]:
df.select.when(
    df.petal_width < 1
)

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|variety|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| Setosa|
|         4.9|        3.0|         1.4|        0.2| Setosa|
|         4.7|        3.2|         1.3|        0.2| Setosa|
|         4.6|        3.1|         1.5|        0.2| Setosa|
|         5.0|        3.6|         1.4|        0.2| Setosa|
+------------+-----------+------------+-----------+-------+
only showing top 5 rows



In [ ]:
for
if petal_width < 1 then Setosa
if 1 <= petal_width < 1.7  then Setosa

In [75]:
df = df.withColumn('predict',
    F.when(df.petal_width < 1, 'Setosa')
    .when((1 <= df.petal_width) & (df.petal_width < 1.7), 'Versicolor')
    .when(1.7 <= df.petal_width, 'Verginia')
)
df.show(5)

+------------+-----------+------------+-----------+-------+-------+
|sepal_length|sepal_width|petal_length|petal_width|variety|predict|
+------------+-----------+------------+-----------+-------+-------+
|         5.1|        3.5|         1.4|        0.2| Setosa| Setosa|
|         4.9|        3.0|         1.4|        0.2| Setosa| Setosa|
|         4.7|        3.2|         1.3|        0.2| Setosa| Setosa|
|         4.6|        3.1|         1.5|        0.2| Setosa| Setosa|
|         5.0|        3.6|         1.4|        0.2| Setosa| Setosa|
+------------+-----------+------------+-----------+-------+-------+
only showing top 5 rows



In [81]:
df = df.withColumn('check', F.when(df.variety == df.predict, 1).otherwise(0))
df.show(5)

+------------+-----------+------------+-----------+-------+-------+-----+
|sepal_length|sepal_width|petal_length|petal_width|variety|predict|check|
+------------+-----------+------------+-----------+-------+-------+-----+
|         5.1|        3.5|         1.4|        0.2| Setosa| Setosa|    1|
|         4.9|        3.0|         1.4|        0.2| Setosa| Setosa|    1|
|         4.7|        3.2|         1.3|        0.2| Setosa| Setosa|    1|
|         4.6|        3.1|         1.5|        0.2| Setosa| Setosa|    1|
|         5.0|        3.6|         1.4|        0.2| Setosa| Setosa|    1|
+------------+-----------+------------+-----------+-------+-------+-----+
only showing top 5 rows



In [96]:
accuracy = df.select(F.sum(df.check)).take(1)[0][0]
f'accuracy = {accuracy / df.count():.2f}'

'accuracy = 0.65'